In [1]:
!pip install pypdf
!pip install python-dotenv

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [3]:
!pip install sentence_transformers

In [4]:
!pip install llama-index


In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

In [7]:
documents = SimpleDirectoryReader("/content/Data/").load_data()

In [8]:

from llama_index.prompts.prompts import SimpleInputPrompt

system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided. You have to give the uses of the plant name input by the user along with extra information related to it."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [10]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-cr

In [11]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


In [13]:
service_context = ServiceContext.from_defaults(
    chunk_size=2048,
    llm=llm,
    embed_model=embed_model
)

In [14]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [16]:
query_engine = index.as_query_engine()


In [17]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

tell me the uses of tulsi


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Tulsi, also known as holy basil, has a wide range of uses. According to the provided context information, some of the uses of tulsi include:

* Analgesic: Tulsi has been traditionally used to reduce pain and inflammation.
* Antibacterial: Tulsi has antibacterial properties and can be used to treat bacterial infections.
* Anti-inflammatory: Tulsi has anti-inflammatory properties and can be used to reduce inflammation.
* Antipyretic: Tulsi can be used to reduce fever.
* Antiseptic: Tulsi has antiseptic properties and can be used to prevent infection.
* Aphrodisiac: Tulsi has been traditionally used to increase sexual desire and male fertility.
* Bitter: Tulsi has a bitter taste and can be used to treat digestive issues.
* Candidicide: Tulsi has been traditionally used to treat candidiasis.
* Cardiotonic: Tulsi has been traditionally used to treat cardiovascular issues.
* Depurative: Tulsi has been traditionally used to
what are the benefits of neem
Neem has numerous benefits. It has been

KeyboardInterrupt: ignored

In [ ]:
import pickle

# get embeddings for each document
embeddings = [embed_model.embed(doc) for doc in documents]

# save embeddings using pickle
with open('embeddings.pickle', 'wb') as f:
    pickle.dump(embeddings, f)
